In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-dark')
%config InlineBackend.figure_format = 'retina'
pd.options.display.max_rows = 20
pd.options.display.max_columns = 20

plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.color'] = 'r'
plt.rcParams['axes.grid'] = True

데이터셋 - 화재통계

In [2]:
fire_data = pd.read_csv('Dataset/화재발생정보.csv', encoding='cp949',index_col='연번')
fdf = fire_data.copy()

fdf.info()
fdf.isnull().sum()
# 읍면동 14/ 부동산/동산 3858

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44178 entries, 1 to 44178
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   사망         44178 non-null  int64  
 1   부상         44178 non-null  int64  
 2   인명피해(명)소계  44178 non-null  int64  
 3   화재발생년월일    44178 non-null  object 
 4   시도         44178 non-null  object 
 5   시군구        44178 non-null  object 
 6   읍면동        44164 non-null  object 
 7   재산피해소계     44178 non-null  int64  
 8   부동산        40320 non-null  float64
 9   동산         40320 non-null  float64
 10  장소대분류      44178 non-null  object 
 11  장소중분류      44178 non-null  object 
 12  장소소분류      44178 non-null  object 
dtypes: float64(2), int64(4), object(7)
memory usage: 4.7+ MB


사망              0
부상              0
인명피해(명)소계       0
화재발생년월일         0
시도              0
시군구             0
읍면동            14
재산피해소계          0
부동산          3858
동산           3858
장소대분류           0
장소중분류           0
장소소분류           0
dtype: int64

결측값 처리

In [3]:
fdf = fdf.drop(['부동산','동산'],axis=1)

fdf['읍면동'].isnull().sum()
fdf_null = fdf[fdf['읍면동'].isnull()==1]
fdf_null # 모두 경기도 부천시 <<
fdf = fdf.drop(fdf_null.index)

In [4]:
fdf_seoul = fdf[fdf['시도']=='서울특별시']

In [5]:
fdf

,사망,부상,인명피해(명)소계,화재발생년월일,시도,시군구,읍면동,재산피해소계,장소대분류,장소중분류,장소소분류
연번,,,,,,,,,,,
1,0,0,0,2017-01-01 00:00,서울특별시,강남구,논현동,13391,주거,단독주택,상가주택(주택부분에서 화재가 발생한 경우에만 해당)
2,0,0,0,2017-01-01 00:09,경상북도,문경시,흥덕동,195,생활서비스,음식점,한식
3,0,0,0,2017-01-01 00:14,경상남도,거제시,일운면,22,기타,야외,기타야외
4,0,0,0,2017-01-01 00:57,경기도,안산시상록구,본오동,1,주거,단독주택,다가구주택
5,0,0,0,2017-01-01 00:59,전라북도,익산시,인화동1가,966,"판매,업무시설",숙박시설,모텔
...,...,...,...,...,...,...,...,...,...,...,...
44174,0,0,0,2017-12-31 22:54,강원도,동해시,망상동,0,기타,야외,기타야외
44175,0,0,0,2017-12-31 23:26,서울특별시,서대문구,현저동,5255,주거,단독주택,단독주택
44176,0,0,0,2017-12-31 23:35,광주광역시,북구,유동,851,"의료,복지시설",의료시설,병원


In [6]:
fdf_seoul_location = fdf_seoul[['시군구','장소대분류','장소중분류']]

In [7]:
fdf_seoul_location

,시군구,장소대분류,장소중분류
연번,,,
1,강남구,주거,단독주택
20,마포구,주거,단독주택
21,강서구,주거,공동주택
45,강동구,주거,공동주택
47,강북구,기타,야외
...,...,...,...
44128,양천구,생활서비스,음식점
44136,성동구,주거,단독주택
44137,노원구,주거,공동주택


In [9]:
fire_loc_seoul = pd.pivot_table(fdf_seoul_location,index='시군구',columns='장소대분류',values='장소중분류',aggfunc='count',fill_value=0)
fire_loc_seoul

장소대분류,교육시설,기타,기타서비스,산업시설,생활서비스,운수자동차시설,"위험물,가스제조소","의료,복지시설",임야,"자동차,철도차량",주거,집합시설,"판매,업무시설"
시군구,,,,,,,,,,,,,
강남구,7,67,12,2,109,3,0,17,0,32,160,2,91
강동구,4,35,0,8,34,2,0,7,3,20,123,2,31
강북구,1,36,2,1,17,1,0,2,0,8,100,1,17
강서구,3,39,8,9,37,4,0,3,4,38,183,5,31
관악구,9,25,0,4,60,3,1,2,9,13,127,1,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
용산구,1,31,1,8,38,1,0,5,1,6,57,3,15
은평구,4,11,3,1,22,1,0,6,2,15,84,1,9
종로구,2,39,14,21,42,1,0,3,1,16,63,6,26


In [10]:
fire_loc_seoul.to_csv('서울화재빈도데이터.csv')